In [1]:
class Flight:
    def __init__(self, start, dest, cost, start_time, landing_time, capacity):
        self.start = start
        self.dest = dest
        self.cost = cost
        self.start_time = start_time
        self.landing_time = landing_time
        self.capacity = capacity

In [2]:
class Passenger:
    def __init__(self, booking_id, name, dest, planned_departure_time, planned_arrival_time, flight_distance):
        self.booking_id = booking_id
        self.name = name
        self.dest = dest
        self.planned_departure_time = planned_departure_time
        self.planned_arrival_time = planned_arrival_time
        self.flight_distance = flight_distance

In [3]:
def get_node_info_from_index(node_index, sg, nsg):
    """ For a given qubit index, returns the corresponding time slice and the node label. """

    for time_slice in range(len(nsg)):
        if node_index in nsg[time_slice]:

            pos = nsg[time_slice].index(node_index)
            node_label = sg[time_slice][pos]

            return time_slice, node_label

In [4]:
def result_evaluation(result, offset, sg, nsg, cost_matrix):
    """
    For a given qubit string, returns an explanation of the nodes that are passed per time slice.
    """
    counter = 0
    previous_node = None
    total_cost = 0
    solution_not_valid = False

    for i in range(len(result)):
        if result[i] == '1':
            time_slice, node_label = get_node_info_from_index(i + offset, sg, nsg)
            if time_slice != counter:
                solution_not_valid = True

            if counter == 0:
                previous_node = node_label
            else:
                cost = cost_matrix[previous_node][node_label]
                total_cost += cost
                print(('\t\t\t\tcost: {}').format(cost))
                previous_node = node_label


            print(('node {} at time slice {}').format(node_label, time_slice))

            counter += 1

            
    print(('\ntotal cost: {}').format(total_cost))
    if total_cost >= 500 or solution_not_valid:
        print('\nIt seems like no valid solution could be found. This solution disregards at least one constraint.')

    return

In [5]:
def get_unique_nodes(edge_list):
    """
    Creates a list of all unique nodes appearing in a given graph.
    The start node is added by default; every other node is only included
    if it can be reached from another node. Isolated nodes will not be 
    added to the list, unless they happen to be the start node.
    """
    
    nr_nodes = 0
    unique_nodes = []

    for edge in edge_list:
        if edge[0] not in unique_nodes:
            unique_nodes.append(edge[0])
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes

In [6]:
def construct_sliced_graph(flight_dict, start, dest):
    """
    Constructs a two-dimensional array from a list of edges. Each subarray represents 
    one time slice. The returned array contains the specified start node in the 
    first sub-array; based on this start node, every node that can be reached in 
    n steps is present in the nth subarray. Nodes that can be reached in a different 
    amount of steps are present in every fitting subarray.
    """

    # we define a new dict 'flights' here, this is local
    # gets updated with the zero-cost edge with both start and destination = dest
    flight_dict.update({'Z' + str(dest) : Flight(dest, dest, 0,0,0,0)})
    flights = flight_dict.copy()
    
    # every supplementary flight Z gets removed except for the one relevant here
    for f in list(flights.keys()):
        if f[0] == 'Z' and f[1:] != str(dest):
            del flights[f]


    sg = []
    sg.append([flight for flight in flights if flights[flight].start == start])

    nr_unique_nodes = len(get_unique_nodes(flights))

    for k in range(nr_unique_nodes):
        if len(sg[k]) != 0 and not (len(sg[k]) == 1 and flights[sg[k][0]].dest == dest):
            sg.append([])
            for f1 in flights:
                for f2 in sg[k]:
                    if flights[f1].start == flights[f2].dest:
                        sg[k+1].append(f1)
                        break 
        else:
            break

    # sort out any flights in the last slice that do not lead to destination
    sg[-1] = [f for f in sg[-1] if flights[f].dest == dest]

    # going back, we can eliminate edges (flights) that won't lead 
    # to places that the flights in the next slice depart from, 
    # a.k.a. dead ends

    for i in range(len(sg) - 1):
        edges_to_remove = []
        index = len(sg) - 1 - i  # since we count backwards

        for j in sg[index - 1]:
            leads_to_next_slice = 0  # number of edges in the next slice that sg[index-1][j] is connected to
            dest_j = flights[j].dest
            for k in sg[index]:

                # for each node in the next slice, add 1 if sg[index-1][j] is connected to it, 0 if not
                if dest_j == flights[k].start:
                    leads_to_next_slice += 1
                

            # if leads_to_next_slice is zero, it means sg[index-1][j] is not connected to any node in the next slice; essentially a dead end. 
            # it is removed, and any node in the previous slice that only connected to it will thus also be a dead end, and will be removed. etc., etc.
            if leads_to_next_slice == 0:
                edges_to_remove.append(j)

        for edge in edges_to_remove:
            sg[index-1].remove(edge)

    slices_to_remove = []
    for i in range(len(sg)):
        index = len(sg) - 1 - i

        if len(sg[index]) == 1 and flights[sg[index][0]].dest == dest:
            slices_to_remove.append(index)
        else:
            break
    
    for i in slices_to_remove:
        del sg[i]

    return sg

In [7]:
def construct_numbered_sliced_graph(sg, offset):
    """
    Based on a two-dimensional array that constructed time slices from a graph,
    renumbers the contents by giving each element its index in the corresponding 
    flattened array. Since the numbered sliced graph is used for getting the qubit 
    indices of the nodes, and since there are several sliced graphs, an offset is
    necessary.
    Example: With input ([[0], [1, 3], [2, 3]], 0), output would be [[0], [1, 2], [3, 4]].
    With input ([[0], [1, 3], [2, 3], [4, 3]], 5), output would be [[5], [6, 7], [8, 9], [10, 11]].
    """
    
    nsg = [] # numbered sliced graph
    node_counter = 0

    for m in range(len(sg)):
        nsg.append([])
        for n in range(len(sg[m])):
            nsg[m].append(node_counter + offset)
            node_counter+=1
    
    return nsg, node_counter

In [8]:
def flight_cost(flight):
    """
    Returns the flight cost for a given flight, identified by its flight number.
    """
    
    return flights[flight].cost

In [9]:
def transfer_check(flight_1, flight_2, passenger_index):
    """
    For two given flights, checks if the time inbetween is 
    smaller than the minimum transfer time. If yes, returns 
    the specified penalty value. If no, returns 0. If the
    flights are not connecting flights, returns 0.
    """

    dest_flight_1 = flights[flight_1].dest
    start_flight_2 = flights[flight_2].start

    destination = passengers[passenger_index].dest

    if dest_flight_1 == start_flight_2 and dest_flight_1 == destination:
        return 0
    elif dest_flight_1 != start_flight_2: # the flights are not connecting flights
        return penalty
    else:
        landing_time = flights[flight_1].landing_time
        start_time = flights[flight_2].start_time

        if start_time - landing_time < 1:
            return penalty
        else:
            return 0

In [10]:
def transfer_time(flight_1, flight_2):
    """
    For two given flights, returns the time transfer time.
    If flights aren't connected or the transfer time is 
    negative (second flight starts earlier than first one
    lands), returns 0.
    """

    dest_flight_1 = flights[flight_1].dest
    start_flight_2 = flights[flight_2].start

    if dest_flight_1 != start_flight_2:
        return 0

    landing_time_flight_1 = flights.get(flight_1).landing_time
    start_time_flight_2 = flights.get(flight_2).start_time

    transfer_duration = start_time_flight_2 - landing_time_flight_1

    if transfer_duration < 0:
        return 0
    else:
        return transfer_duration

In [11]:
def connection_check(flight_1, flight_2):
    """
    For two given flights, checks if the destination of
    the first is the start of the second, i.e. if they are
    connecting flights. If yes, return 0, if not, return
    the specified penalty value.
    """

    destination = flights[flight_1].landing_time
    start = flights[flight_2].start_time

    if destination != start:
        return penalty
    else:
        return 0

In [12]:
#def capacity_polynomial(flight, nrpassengers):

#    if flight[0] == 'Z':
#        return 0
#    
#    qubit_indices = []

#    for k in range(len(passengers)):
#        for c in range(len(test_sg[k])):
#            for e in range(len(test_sg[k][c])):
#                if test_sg[k][c][e] == flight:
#                    qubit_indices.append(X[test_nsg[k][c][e]])
    
#    capacity = flights[flight].capacity

#    sum_vars = 0
#    for i in qubit_indices:
#        sum_vars += i

#    global nr_qubits

#    nr_addends = nrpassengers if nrpassengers < capacity else capacity # maybe one could just reference the passenger var
#    nr_addends = nr_addends if nr_addends < len(qubit_indices) else len(qubit_indices)

#    if nr_addends == 0:
#        return 0  # this is okay if all flights with capacity 0 are sorted out at the beginning
#    else:
#        # we need nr_addends - 1 vars
#        polynomial = 0
#        minuends = [0]

#        for i in qubit_indices:
#            minuends[0] += i 

#        for i in range(1, nr_addends):
#            minuends.append(0)
#            for j in range(i, nr_addends):
#                minuends[i] += X[nr_qubits + j - 1]
#        minuends.append(0)

#        print(minuends)
            
#        for i in range(len(minuends) - 1):
#            polynomial += P * ((minuends[i] - (nr_addends - i)) * (minuends[i] - minuends[i + 1]))


#        nr_qubits += nr_addends - 1

#        return polynomial

In [15]:
def waiting_time(passenger, flight):
    wait = flights[flight].start_time - passengers[passenger].planned_departure_time

    # this can be adapted. maybe it makes sense to use the absolute value of wait, not sure
    if wait < 0 or flight[0] == 'Z':
        wait = 0
    
    return wait

In [13]:
def get_qubit_indices(flight):
    qubit_indices = []

    for k in range(len(passengers)):
        for c in range(len(test_sg[k])):
            for e in range(len(test_sg[k][c])):
                if test_sg[k][c][e] == flight:
                    qubit_indices.append(test_nsg[k][c][e])

    return qubit_indices

In [14]:
def add_capacity_constraints(cost_func, flights, nrpassengers):
    for flight in flights:
        if flight[0] != 'Z':

            capacity = flights[flight].capacity
            upper_bound = nrpassengers if nrpassengers < capacity else capacity
            
            qubit_indices = get_qubit_indices(flight)

            if upper_bound > len(qubit_indices):
                upper_bound = len(qubit_indices)

            if len(qubit_indices) > upper_bound:

                # specifies which qubits will be summed up for the constraint. 
                # the coefficients for any qubit are always 1, since we just add them up
                linear_coefficients = {} 

                for q in qubit_indices:
                    qubit_name = 'X_' + str(q)
                    linear_coefficients.update({q: 1})

                cost_func.linear_constraint(linear=linear_coefficients, sense='<=', rhs=upper_bound, name=('capacity_constraint_' + flight))

In [16]:
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt

In [17]:
# data
edge_data = [
    ('A11', 0, 1, 160, 8, 10, 2),
    ('A12', 0, 2, 60, 8.5, 9.5, 2), 
    ('A13', 0, 3, 215, 7.5, 10.5, 1), 
    ('A14', 2, 1, 80, 10.5, 12, 2), 
    ('A15', 2, 3, 75, 10, 11, 2),
    ('A16', 1, 3, 40, 13, 14, 1)
    ]

# cost, without fluggastverordnung:
## passenger 1: A11
## passenger 2: A12, A14
## passenger 3: A12, A14, A16

# cost, with fluggastverordnung:
## passenger 1: A11
## passenger 2: A12, A14
## passenger 3: A13

# time:
## passenger 1: A11
## passenger 2: A11
## passenger 3: A13

# removing flights with 0 capacity
edge_data = [edge for edge in edge_data if edge[6] > 0]

flights = {}

for edge in edge_data:
    flights.update({edge[0] : Flight(*edge[1:])})


# not a dict because we use the passengers' indices in this list rather than a booking id for identification. order is important here
passengers = [
    Passenger("HX7R2W", "Akiko Takeda", 1, 8, 9, 340),
    Passenger("L5D32X", "B", 1, 8.5, 10.5, 400),
    Passenger("N7LR19", "C", 3, 7, 8.5, 610)
]

start_node = 0
penalty = 5000

In [18]:
#display_graph = nx.DiGraph()  # create an nx instance of a directed graph
#edge_labels = {}

#for edge in edge_data:
#    edge_labels.update({(edge[1], edge[2]) : edge[0]})

#    display_graph.add_weighted_edges_from([
#        edge[1:4]
#        ])

In [19]:
test_sg = []
test_nsg = []
nodes_total = 0

for k in range(len(passengers)):
    test_sg.append(construct_sliced_graph(flights, start_node, passengers[k].dest))

    numbered_subgraph, nr_nodes_subgraph = construct_numbered_sliced_graph(test_sg[k], offset=nodes_total)
    test_nsg.append(numbered_subgraph)

    nodes_total += nr_nodes_subgraph

nr_qubits = nodes_total
nr_qubits

17

In [20]:
def fgrvo_penalties(passengers, flights):
    """
    Simplified version of the EU regulations on flight annulations.
    Returns a dict: keys are the passenger index and the flight number,
    value is the penalty. In cases where the flight leads to the passenger's
    destination, the penalty is determined, in any other case, the value is 0.
    """
    fgrvo_dict = {}

    for k in range(len(passengers)):
        passenger = passengers[k]

        for f in flights:
            flight = flights[f]
            if f[0] == "Z":
                penalty = 0
        
            elif passenger.dest == flight.dest:
                if passenger.flight_distance <= 1500:
                    if flight.landing_time - passenger.planned_arrival_time <= 2:
                        penalty = 125
                    else:
                        penalty = 250
                elif passenger.flight_distance <= 3500:
                    if flight.landing_time - passenger.planned_arrival_time <= 3:
                        penalty = 200
                    else:
                        penalty = 400
                elif passenger.flight_distance > 3500:
                    if flight.landing_time - passenger.planned_arrival_time <= 4:
                        penalty = 300
                    else:
                        penalty = 600
                else:
                    penalty = 0
                
            else:
                penalty = 0
                
                
            fgrvo_dict.update({ fgrvo[k, f] : penalty })

    return fgrvo_dict

In [21]:
k = sym.symbols('k') # passengers
c = sym.symbols('c') # slices
e = sym.symbols('e') # edges
f = sym.symbols('f') # edges
X = sym.IndexedBase('X') # binary vars for qubits
P = sym.symbols('P') # penalty

edge1 = sym.symbols('edge1') # edges
edge2 = sym.symbols('edge2') # edges

nrpassengers = sym.symbols('nrpassengers')
nrflights = sym.symbols('nrflights')

nrslices = sym.IndexedBase('nrslices')
nredges = sym.IndexedBase('nredges')

sg = sym.IndexedBase('sg')
nsg = sym.IndexedBase('nsg')

validtransfer = sym.IndexedBase('validtransfer')
transfertime = sym.IndexedBase('transfertime')
connection = sym.IndexedBase('connection')
cost = sym.IndexedBase('cost')
flightduration = sym.IndexedBase('flightduration')
capacity = sym.IndexedBase('capacity')
fgrvo = sym.IndexedBase('fgrvo')
waitingtime = sym.IndexedBase('waitingtime')

flightlist = sym.IndexedBase('flightlist')

In [22]:
# only one flight in each time slice
# cost of flights
# enough transfer time
# fgrvo!
cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
            ) - 1)**2
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * cost[sg[k,c,e]] 
    
            , (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    )
    +
    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])
                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    )
    
    
    , (k, 0, nrpassengers-1) 
)
cost_function

Sum(Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + 2*Sum(0.5*Sum(X[nsg[k, c, e]]*cost[sg[k, c, e]], (e, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 1)) + Sum(2*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*validtransfer[sg[k, c, edge1], sg[k, c + 1, edge2], k], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrpassengers - 1))

In [23]:
# only one flight in each time slice
# cost of flights
# enough transfer time
# fgrvo
cost_function_fgrvo = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
            ) - 1)**2
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * cost[sg[k,c,e]] 
    
            , (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * fgrvo[k, sg[k,c,e]]
    
            , (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    )   
    +

    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])
                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    )
    
    , (k, 0, nrpassengers-1) 
)
cost_function

Sum(Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + 2*Sum(0.5*Sum(X[nsg[k, c, e]]*cost[sg[k, c, e]], (e, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 1)) + Sum(2*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*validtransfer[sg[k, c, edge1], sg[k, c + 1, edge2], k], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrpassengers - 1))

In [24]:
# takes total duration per passenger as metric instead of cost
cost_function_time = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
            ) - 1)**2
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * flightduration[sg[k,c,e]], (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    ) 
    + 

    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])

                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    ) + 

    sym.Sum( 200 *
        sym.Sum( 0.5 *
            sym.Sum(
                X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * transfertime[sg[k,c,edge1], sg[k,c+1,edge2]]

                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    ) +
    sym.Sum( X[nsg[k, 0, e]] * waitingtime[k, sg[k, 0, e]]  , (e, 0, nredges[k, 0] - 1))
    
    
    , (k, 0, nrpassengers-1) 
)
cost_function_time

Sum(Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + Sum(X[nsg[k, 0, e]]*waitingtime[k, sg[k, 0, e]], (e, 0, nredges[k, 0] - 1)) + 2*Sum(0.5*Sum(X[nsg[k, c, e]]*flightduration[sg[k, c, e]], (e, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 1)) + Sum(200*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*transfertime[sg[k, c, edge1], sg[k, c + 1, edge2]], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)) + Sum(2*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*validtransfer[sg[k, c, edge1], sg[k, c + 1, edge2], k], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrpassengers - 1))

In [25]:
single_values_dict = {
    P: penalty,
    nrpassengers: len(passengers), 
    nrflights: len(flights)
    }

nsg_dict = {
    nsg[k,c,e]: test_nsg[k][c][e] 
    for k in range(len(passengers)) 
    for c in range(len(test_nsg[k])) 
    for e in range(len(test_nsg[k][c]))
}

sg_dict = {
    sg[k,c,e]: test_sg[k][c][e] 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k])) 
    for e in range(len(test_sg[k][c]))
}

nr_slices_dict = {
    nrslices[k]: len(test_sg[k]) 
    for k in range(len(passengers))
}

nr_edges_dict = {
    nredges[k,c]: len(test_sg[k][c]) 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k]))
}

cost_dict = {
    cost[edge]: flight_cost(edge)
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k])) 
    for edge in test_sg[k][c]
}

valid_transfer_dict = {
    validtransfer[edge1, edge2, k]: transfer_check(edge1, edge2, k) 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k]) - 1) 
    for edge1 in test_sg[k][c] 
    for edge2 in test_sg[k][c+1]
}

transfer_time_dict = {
    transfertime[edge1, edge2]: transfer_time(edge1, edge2) 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k]) - 1) 
    for edge1 in test_sg[k][c] 
    for edge2 in test_sg[k][c+1]
}

flight_duration_dict = {
    flightduration[f]: flights[f].landing_time - flights[f].start_time
    for f in flights
}

fgrvo_dict = fgrvo_penalties(passengers, flights)

waiting_time_dict = {
    waitingtime[k, edge] : waiting_time(k, edge)
    for k in range(len(passengers))
    for edge in test_sg[k][0]
}

#flight_list_dict = {
#    flightlist[f]: flight_list[f]
#    for f in range(len(flights))
#}

In [26]:
waiting_time_dict

{waitingtime[0, A11]: 0,
 waitingtime[0, A12]: 0.5,
 waitingtime[1, A11]: 0,
 waitingtime[1, A12]: 0.0,
 waitingtime[2, A11]: 1,
 waitingtime[2, A12]: 1.5,
 waitingtime[2, A13]: 0.5}

In [29]:
# depending on the metric, either cost_function, cost_function_fgrvo or cost_function_time can be chosen here

start = time.time()
cost_poly = sym.Poly(cost_function_time
                     .subs(single_values_dict)
                     .doit()
                     .subs(nr_slices_dict)
                     .doit()
                     .subs(nr_edges_dict)
                     .doit()
                     .subs(sg_dict)
                     .subs(nsg_dict)
                     .doit()
                     .subs(valid_transfer_dict)
                     .subs(transfer_time_dict)
                     .subs(cost_dict)
                     .subs(flight_duration_dict)
                     .subs(fgrvo_dict)
                     .subs(waiting_time_dict)
                     .doit()
                     .subs(single_values_dict)
                     ,
                     [X[i] for i in range(nr_qubits)])
end = time.time()
print(end-start)
cost_poly

22.945146083831787


Poly(5000.0*X[0]**2 + 10000.0*X[0]*X[1] + 5000.0*X[0]*X[2] - 9998.0*X[0] + 5000.0*X[1]**2 + 100.0*X[1]*X[2] + 5000.0*X[1]*X[3] - 9998.5*X[1] + 5000.0*X[2]**2 + 10000.0*X[2]*X[3] - 9998.5*X[2] + 5000.0*X[3]**2 - 10000.0*X[3] + 5000.0*X[4]**2 + 10000.0*X[4]*X[5] + 5000.0*X[4]*X[6] - 9998.0*X[4] + 5000.0*X[5]**2 + 100.0*X[5]*X[6] + 5000.0*X[5]*X[7] - 9999.0*X[5] + 5000.0*X[6]**2 + 10000.0*X[6]*X[7] - 9998.5*X[6] + 5000.0*X[7]**2 - 10000.0*X[7] + 5000.0*X[8]**2 + 10000.0*X[8]*X[9] + 10000.0*X[8]*X[10] + 5000.0*X[8]*X[11] + 5000.0*X[8]*X[12] + 300.0*X[8]*X[13] + 5000.0*X[8]*X[14] - 9997.0*X[8] + 5000.0*X[9]**2 + 10000.0*X[9]*X[10] + 100.0*X[9]*X[11] + 5050.0*X[9]*X[12] + 5000.0*X[9]*X[13] + 5000.0*X[9]*X[14] - 9997.5*X[9] + 5000.0*X[10]**2 + 5000.0*X[10]*X[11] + 5000.0*X[10]*X[12] + 5000.0*X[10]*X[13] - 9996.5*X[10] + 5000.0*X[11]**2 + 10000.0*X[11]*X[12] + 10000.0*X[11]*X[13] + 10000.0*X[11]*X[14] + 100.0*X[11]*X[15] + 5000.0*X[11]*X[16] - 9998.5*X[11] + 5000.0*X[12]**2 + 10000.0*X[12]*X[1

In [30]:
import qiskit
from qiskit.algorithms import QAOA

from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram

# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables 
for i in range(nr_qubits):#nrqubits in range here
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits)
        for j in range(i,nr_qubits)
    }
    )

print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9998 X_0 - 9998 X_1 - 9998 X_2 - 10000 X_3 - 9998 X_4 - 9999 X_5
      - 9998 X_6 - 10000 X_7 - 9997 X_8 - 9997 X_9 - 9996 X_10 - 9998 X_11
      - 9999 X_12 - 9999 X_13 - 10000 X_14 - 9999 X_15 - 10000 X_16 + [
      10000 X_0^2 + 20000 X_0*X_1 + 10000 X_0*X_2 + 10000 X_1^2 + 200 X_1*X_2
      + 10000 X_1*X_3 + 10000 X_2^2 + 20000 X_2*X_3 + 10000 X_3^2 + 10000 X_4^2
      + 20000 X_4*X_5 + 10000 X_4*X_6 + 10000 X_5^2 + 200 X_5*X_6
      + 10000 X_5*X_7 + 10000 X_6^2 + 20000 X_6*X_7 + 10000 X_7^2 + 10000 X_8^2
      + 20000 X_8*X_9 + 20000 X_8*X_10 + 10000 X_8*X_11 + 10000 X_8*X_12
      + 600 X_8*X_13 + 10000 X_8*X_14 + 10000 X_9^2 + 20000 X_9*X_10
      + 200 X_9*X_11 + 10100 X_9*X_12 + 10000 X_9*X_13 + 10000 X_9*X_14
      + 10000 X_10^2 + 10000 X_10*X_11 + 10000 X_10*X_12 + 10000 X_10*X_13
      + 10000 X_11^2 + 20000 X_11*X_12 + 20000 X_11*X_13 + 20000 X_11*X_14
      + 200 X_11*

In [31]:
test_sg

[[['A11', 'A12'], ['A14', 'Z1']],
 [['A11', 'A12'], ['A14', 'Z1']],
 [['A11', 'A12', 'A13'], ['A14', 'A15', 'A16', 'Z3'], ['A16', 'Z3']]]

In [32]:
add_capacity_constraints(qiskit_cost_function, flights, len(passengers))

In [33]:
print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9998 X_0 - 9998 X_1 - 9998 X_2 - 10000 X_3 - 9998 X_4 - 9999 X_5
      - 9998 X_6 - 10000 X_7 - 9997 X_8 - 9997 X_9 - 9996 X_10 - 9998 X_11
      - 9999 X_12 - 9999 X_13 - 10000 X_14 - 9999 X_15 - 10000 X_16 + [
      10000 X_0^2 + 20000 X_0*X_1 + 10000 X_0*X_2 + 10000 X_1^2 + 200 X_1*X_2
      + 10000 X_1*X_3 + 10000 X_2^2 + 20000 X_2*X_3 + 10000 X_3^2 + 10000 X_4^2
      + 20000 X_4*X_5 + 10000 X_4*X_6 + 10000 X_5^2 + 200 X_5*X_6
      + 10000 X_5*X_7 + 10000 X_6^2 + 20000 X_6*X_7 + 10000 X_7^2 + 10000 X_8^2
      + 20000 X_8*X_9 + 20000 X_8*X_10 + 10000 X_8*X_11 + 10000 X_8*X_12
      + 600 X_8*X_13 + 10000 X_8*X_14 + 10000 X_9^2 + 20000 X_9*X_10
      + 200 X_9*X_11 + 10100 X_9*X_12 + 10000 X_9*X_13 + 10000 X_9*X_14
      + 10000 X_10^2 + 10000 X_10*X_11 + 10000 X_10*X_12 + 10000 X_10*X_13
      + 10000 X_11^2 + 20000 X_11*X_12 + 20000 X_11*X_13 + 20000 X_11*X_14
      + 200 X_11*

In [34]:
# execute QAOA on local simulator
qaoa = QAOA(quantum_instance =
             QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator')))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)

results = []

for i in range(1):
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)

In [35]:
results[0]

optimal function value: -24790.0
optimal value: [1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1.]
status: SUCCESS

In [36]:
optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None

results_classic = []

for i in range(2):
    result = optimizer.solve(qiskit_cost_function)
    results_classic.append(result)

In [37]:
results_classic

[optimal function value: -34992.0
 optimal value: [ 1. -0.  0.  1.  1.  0.  0.  1. -0.  0.  1. -0.  0. -0.  1.  0.  1.]
 status: SUCCESS,
 optimal function value: -34992.0
 optimal value: [ 1. -0.  0.  1.  1.  0.  0.  1. -0.  0.  1. -0.  0. -0.  1.  0.  1.]
 status: SUCCESS]

In [38]:
test_sg

[[['A11', 'A12'], ['A14', 'Z1']],
 [['A11', 'A12'], ['A14', 'Z1']],
 [['A11', 'A12', 'A13'], ['A14', 'A15', 'A16', 'Z3'], ['A16', 'Z3']]]

In [39]:
qubit_results = []
for result in results:
    qubit_string = str(result.x).replace(' ', '').replace('.', '').replace('-', '')[1:-1]
    qubit_results.append(qubit_string)
qubit_results

['10011001010100011']

In [40]:
def interpret_results(result, offset):
    for i in range(len(result)):
        return
        # qubit index is i + offset
        # find the nsg entry equal to qubit index
        # then get the sg entry
        # and return the flight information


- result for cost without fgrvo:
- optimal function value: -34520.0
- optimal value: [ 1.  0.  0.  1.  0.  1.  1.  0. -0.  1. -0.  1.  0.  0.  0.  1.  0.]
- status: SUCCESS

- result for cost with fgrvo:
- optimal function value: -34112.0
- optimal value: [ 1.  0.  0.  1.  0.  1.  1. -0. -0.  0.  1. -0.  0. -0.  1.  0.  1.]
- status: SUCCESS

- result for time cost:
- optimal function value: -34593.0
- optimal value: [ 1.  0.  0.  1.  1.  0.  0.  1.  0.  0.  1.  0. -0.  0.  1.  0.  1.]
- status: SUCCESS